# anthropic hh-rlhf dataset processing

In [ ]:
from datasets import load_dataset
import re, json
from transformers import AutoTokenizer
import pandas as pd

In [ ]:
hhrlhf = load_dataset("Anthropic/hh-rlhf")
t = AutoTokenizer.from_pretrained("meta-llama/Llama-2-70b-chat-hf")


In [ ]:
dialogs = []
fail_counter = 0
for s in hhrlhf["test"]["chosen"]:
    fields = re.split("(Human:|Assistant:)", s)
    try:
        dialog = []
        assert fields[0].strip() == ""
        assert all(ss.startswith("Human:") for ss in fields[1::4])
        assert all(ss.startswith("Assistant:") for ss in fields[3::4])
        for i in range(2, len(fields) - 1, 2):
            dialog.append(
                {
                    "role": "assistant" if i % 4 == 0 else "user",
                    "content": fields[i].strip(),
                }
            )
        assert len(dialog) >= 1
        assert dialog[-1]["role"] == "user"
        dialog_raw = " ".join([d["content"] for d in dialog])
        num_tokens = len(t(dialog_raw)["input_ids"])
        assert num_tokens <= 200
        dialogs.append(dialog)
    except:
        fail_counter += 1


In [ ]:
pd.DataFrame({"prompt": dialogs}).to_json("../data/rr-rlhf.jsonl", lines=True, orient="records")